# Test on FLCHAIN

In [29]:
import numpy

In [30]:
import sklearn

In [31]:
import survwrap

In [32]:
survwrap.list_available_datasets()

('flchain', 'gbsg2', 'metabric', 'support')

In [33]:
mb_df = survwrap.get_data('flchain')
mb_df.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6524 entries, 0 to 6523
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         6524 non-null   float64
 1   sex         6524 non-null   float64
 2   sample.yr   6524 non-null   int64  
 3   kappa       6524 non-null   float64
 4   lambda      6524 non-null   float64
 5   flc.grp     6524 non-null   int64  
 6   creatinine  6524 non-null   float64
 7   mgus        6524 non-null   float64
 8   time        6524 non-null   float64
 9   event       6524 non-null   float64
dtypes: float64(8), int64(2)
memory usage: 560.7 KB


In [34]:
X, y = mb_df.get_X_y()
X.shape, y.shape

((6524, 8), (6524,))

In [35]:
y[:10]

array([( True,   85.), ( True, 1281.), ( True,   69.), ( True,  115.),
       ( True, 1039.), ( True, 1355.), ( True, 2851.), ( True,  372.),
       ( True, 3309.), ( True, 1326.)],
      dtype=[('event', '?'), ('time', '<f8')])

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [36]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [37]:
X_train, X_test, y_train, y_test = survwrap.survival_train_test_split(X, y, rng_seed=2309)

In [38]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((4893, 8), (1631, 8))

In [39]:
survwrap.get_indicator(y).sum(), survwrap.get_indicator(y_train).sum(), survwrap.get_indicator(y_test).sum(),


(1962, 1472, 490)

## check possible dimensionality reduction

In [40]:
from sklearn.decomposition import PCA

In [41]:
pca= PCA(n_components=0.995, random_state=2308).fit(X_train)
print('PCA components:',pca.n_components_)

PCA components: 8


Massive (50%) reduction using PCA

In [42]:
## Stratified CV spliter for survival analysis

In [43]:
#from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold

In [44]:
#testkf= RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=2307)
#for trn,tst in testkf.split(X_train, survwrap.get_indicator(y_train)):
#    print(trn,tst) 

# test coxnet

In [45]:
rng_seed=2401

In [46]:
coxnet = survwrap.CoxNet(rng_seed)
coxnet.fit(X_train, y_train)

CoxNet(rng_seed=2401, alpha=None, l1_ratio=0.5)

In [47]:
coxnet.score(X_test, y_test)

0.7927079761717836

In [48]:
#from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [49]:
cox_grid = survwrap.CoxNet().get_parameter_grid()
cox_grid['l1_ratio'] = [0.5]
cox_grid

{'alpha': [0.001,
  0.003,
  0.005,
  0.008,
  0.01,
  0.02,
  0.03,
  0.04,
  0.05,
  0.06,
  0.07,
  0.08,
  0.09,
  0.1,
  0.15,
  0.2,
  0.3],
 'l1_ratio': [0.5]}

In [50]:
opt_coxnet, opt_coxnet_params, opt_coxnet_search = survwrap.optimize(survwrap.CoxNet(rng_seed), X_train, y_train, 
                                                                    user_grid=cox_grid, n_jobs=4)
opt_coxnet.score(X_test, y_test), opt_coxnet_params

(0.793126155601201, {'alpha': 0.02, 'l1_ratio': 0.5})

In [51]:
if opt_coxnet_search.scoring: print('multi score') 
else: print('Single score')

Single score


In [52]:
#opt_coxnet_search.cv_results_
survwrap.get_model_scores_df(opt_coxnet_search)

['rank_test_score', 'mean_test_score', 'std_test_score', 'params', 'mean_fit_time', 'std_fit_time']


,rank_test_score,mean_test_score,std_test_score,params,mean_fit_time,std_fit_time
5,1,0.791243,0.011211,"{'alpha': 0.02, 'l1_ratio': 0.5}",0.476548,0.050124
6,2,0.791213,0.011123,"{'alpha': 0.03, 'l1_ratio': 0.5}",0.48222,0.030772
4,3,0.791023,0.011584,"{'alpha': 0.01, 'l1_ratio': 0.5}",0.452358,0.015244
3,4,0.791018,0.011652,"{'alpha': 0.008, 'l1_ratio': 0.5}",0.449075,0.010527
0,5,0.790998,0.011742,"{'alpha': 0.001, 'l1_ratio': 0.5}",0.446674,0.011657
1,5,0.790998,0.011742,"{'alpha': 0.003, 'l1_ratio': 0.5}",0.448055,0.010208
2,5,0.790998,0.011742,"{'alpha': 0.005, 'l1_ratio': 0.5}",0.443161,0.009915
7,8,0.790919,0.011149,"{'alpha': 0.04, 'l1_ratio': 0.5}",0.47715,0.043137
8,9,0.790656,0.011157,"{'alpha': 0.05, 'l1_ratio': 0.5}",0.459422,0.022498
9,10,0.790189,0.011183,"{'alpha': 0.06, 'l1_ratio': 0.5}",0.445735,0.006774


# Test FastCPH

In [53]:
fl_lasso=survwrap.FastCPH(rng_seed)
fl_lasso

FastCPH(rng_seed=2401, layer_sizes=[10, 10], tie_approximation='efron', lambda_seq=[0.001, 0.0010249999999999999, 0.001050625, 0.0010768906249999997, 0.0011038128906249996, 0.0011314082128906244, 0.00115969341821289, 0.0011886857536682123, 0.0012184028975099172, 0.0012488629699476653, 0.0012800845441963567, 0.0013120866578012656, 0.0013448888242462971, 0.0013785110448524542, 0.0014129738209737656, 0.0014482981664981096, 0.0014845056206605623, 0.001521618261177076, 0.001559658717706503, 0.0015986501856491654, 0.0016386164402903943, 0.0016795818512976542, 0.0017215713975800952, 0.0017646106825195975, 0.0018087259495825872, 0.0018539440983221518, 0.0019002927007802054, 0.0019478000182997105, 0.001996495018757203, 0.002046407394226133, 0.002097567579081786, 0.0021500067685588306, 0.002203756937772801, 0.0022588508612171212, 0.0023153221327475485, 0.0023732051860662373, 0.002432535315717893, 0.00249334869861084, 0.002555682416076111, 0.0026195744764780132, 0.0026850638383899635, 0.0027521904343497123, 0.0028209951952084546, 0.002891520075088666, 0.0029638080769658825, 0.0030379032788900293, 0.0031138508608622795, 0.0031916971323838364, 0.003271489560693432, 0.0033532767997107677, 0.0034371087197035363, 0.003523036437696125, 0.0036111123486385274, 0.0037013901573544903, 0.0037939249112883522, 0.003888773034070561, 0.003985992359922324, 0.004085642168920382, 0.004187783223143391, 0.0042924778037219755, 0.004399789748815024, 0.0045097844925354, 0.004622529104848784, 0.004738092332470004, 0.004856544640781753, 0.004977958256801297, 0.005102407213221329, 0.0052299673935518615, 0.005360716578390657, 0.005494734492850423, 0.005632102855171683, 0.005772905426550975, 0.005917228062214749, 0.006065158763770117, 0.0062167877328643695, 0.006372207426185978, 0.006531512611840627, 0.006694800427136643, 0.006862170437815058, 0.007033724698760434, 0.007209567816229444, 0.00738980701163518, 0.007574552186926058, 0.007763915991599209, 0.007958013891389188, 0.008156964238673917, 0.008360888344640766, 0.008569910553256782, 0.008784158317088201, 0.009003762275015407, 0.009228856331890791, 0.00945957774018806, 0.00969606718369276, 0.009938468863285078, 0.010186930584867204, 0.010441603849488884, 0.010702643945726105, 0.010970210044369256, 0.011244465295478486, 0.011525576927865449, 0.011813716351062082, 0.012109059259838635, 0.0124117857413346, 0.012722080384867962, 0.01304013239448966, 0.013366135704351901, 0.013700289096960697, 0.014042796324384713, 0.014393866232494331, 0.014753712888306687, 0.015122555710514353, 0.01550061960327721, 0.01588813509335914, 0.016285338470693116, 0.016692471932460443, 0.017109783730771953, 0.017537528324041252, 0.01797596653214228, 0.018425365695445835, 0.018885999837831978, 0.019358149833777778, 0.019842103579622218, 0.020338156169112772, 0.02084661007334059, 0.021367775325174105, 0.021901969708303454, 0.02244951895101104, 0.023010756924786314, 0.023586025847905968, 0.024175676494103616, 0.0247800684064562, 0.025399570116617608, 0.026034559369533045, 0.02668542335377137, 0.02735255893761565, 0.02803637291105604, 0.02873728223383244, 0.029455714289678244, 0.0301921071469202, 0.030946909825593203, 0.03172058257123303, 0.03251359713551385, 0.0333264370639017, 0.03415959799049924, 0.035013587940261715, 0.035888927638768255, 0.03678615082973745, 0.03770580460048089, 0.03864844971549291, 0.039614660958380225, 0.04060502748233973, 0.04162015316939822, 0.042660656998633174, 0.043727173423599, 0.04482035275918896, 0.045940861578168686, 0.0470893831176229, 0.04826661769556347, 0.04947328313795255, 0.05071011521640136, 0.051977868096811396, 0.053277314799231666, 0.054609247669212456, 0.05597447886094276, 0.05737384083246633, 0.05880818685327798, 0.060278391524609926, 0.06178535131272517, 0.0633299850955433, 0.06491323472293187, 0.06653606559100515, 0.06819946723078028, 0.06990445391154979, 0.07165206525933851, 0.07344336689082198, 0.07527945106309253, 0.07716143733966983, 0.07909047327316157, 0.0810677351049906, 0.0830944284826153

In [54]:
#fl_dsm.layer_sizes=[3]
#fl_dsm.learning_rate=0.01
fl_lasso_xplore=fl_lasso.fit(X_train,y_train)
fl_lasso_xplore.score(X_train,y_train)

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/lassonet/utils.py:28: UserWarning: scatter_reduce() is in beta and the API may change at any time. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1664817693763/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1550.)
  return torch.empty(output_size, device=input.device).scatter_reduce(


epoch: 0
loss: 8.593567848205566
epoch: 1
loss: 8.584389686584473
epoch: 2
loss: 8.575233459472656
epoch: 3
loss: 8.5661039352417
epoch: 4
loss: 8.556997299194336
epoch: 5
loss: 8.547910690307617
epoch: 6
loss: 8.538849830627441
epoch: 7
loss: 8.529813766479492
epoch: 8
loss: 8.520800590515137
epoch: 9
loss: 8.51181411743164
epoch: 10
loss: 8.502851486206055
epoch: 11
loss: 8.493910789489746
epoch: 12
loss: 8.484992027282715
epoch: 13
loss: 8.476089477539062
epoch: 14
loss: 8.46721076965332
epoch: 15
loss: 8.458356857299805
epoch: 16
loss: 8.449514389038086
epoch: 17
loss: 8.440692901611328
epoch: 18
loss: 8.431884765625
epoch: 19
loss: 8.42309284210205
epoch: 20
loss: 8.414307594299316
epoch: 21
loss: 8.40553092956543
epoch: 22
loss: 8.396761894226074
epoch: 23
loss: 8.387993812561035
epoch: 24
loss: 8.379231452941895
epoch: 25
loss: 8.370476722717285
epoch: 26
loss: 8.361723899841309
epoch: 27
loss: 8.35296630859375
epoch: 28
loss: 8.344212532043457
epoch: 29
loss: 8.33544921875
epoc

0.7942605159620818

In [63]:
fl_grid =fl_lasso.get_parameter_grid(max_width=X_train.shape[1])
fl_grid

{'layer_sizes': [[8], [8, 8], [8, 8, 8], [8, 8, 8, 8]]}

In [56]:
# Stratified CV
#opt_dsm, opt_dsm_params, opt_dsm_search = optimize(survwrap.DeepSurvivalMachines(rng_seed=2308),  X_train, y_train, n_jobs=8,
                                                  # user_grid=grid,cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=2308).split(X_train,survwrap.get_indicator(y_train)))
#opt_dsm.score(X_test, y_test), opt_dsm_params

In [64]:
# Stratified CV
fl_lasso_cv=survwrap.FastCPH(rng_seed)
#fl_grid=fl_lasso_cv.get_parameter_grid(max_width=X_train.shape[1])
fl_grid['layer_sizes']=[[8]*_ for _ in range(2,5)]
fl_grid

{'layer_sizes': [[8, 8], [8, 8, 8], [8, 8, 8, 8]]}

In [65]:
opt_lasso, opt_lasso_params, opt_lasso_search = survwrap.optimize(fl_lasso_cv,  X_train, y_train, 
                                                                  user_grid=fl_grid,
                                                                 cv=survwrap.survival_crossval_splitter(X_train,y_train,
                                                                                                        n_repeats=1,n_splits=3),
                                                                 )

epoch: 0
loss: 7.954215049743652
epoch: 1
loss: 7.949530124664307
epoch: 2
loss: 7.944864749908447
epoch: 3
loss: 7.940215587615967
epoch: 4
loss: 7.935583114624023
epoch: 5
loss: 7.93096923828125
epoch: 6
loss: 7.926385402679443
epoch: 7
loss: 7.921855449676514
epoch: 8
loss: 7.917341709136963
epoch: 9
loss: 7.912844181060791
epoch: 10
loss: 7.9083662033081055
epoch: 11
loss: 7.903908729553223
epoch: 12
loss: 7.899469375610352
epoch: 13
loss: 7.895051002502441
epoch: 14
loss: 7.890653133392334
epoch: 15
loss: 7.886277675628662
epoch: 16
loss: 7.8819260597229
epoch: 17
loss: 7.877594947814941
epoch: 18
loss: 7.87328577041626
epoch: 19
loss: 7.869001865386963
epoch: 20
loss: 7.864734649658203
epoch: 21
loss: 7.860491752624512
epoch: 22
loss: 7.856273651123047
epoch: 23
loss: 7.852082252502441
epoch: 24
loss: 7.8479204177856445
epoch: 25
loss: 7.843785285949707
epoch: 26
loss: 7.839672088623047
epoch: 27
loss: 7.835582256317139
epoch: 28
loss: 7.831511497497559
epoch: 29
loss: 7.82745695

In [66]:
survwrap.get_model_scores_df(opt_lasso_search)

['rank_test_score', 'mean_test_score', 'std_test_score', 'params', 'mean_fit_time', 'std_fit_time']


,rank_test_score,mean_test_score,std_test_score,params,mean_fit_time,std_fit_time
2,1,0.792478,0.013033,"{'layer_sizes': [8, 8, 8, 8]}",19.106784,0.668043
1,2,0.791197,0.014476,"{'layer_sizes': [8, 8, 8]}",17.164177,0.206789
0,3,0.790994,0.011804,"{'layer_sizes': [8, 8]}",16.022369,2.096882


In [72]:
survwrap.get_model_scores_df(opt_lasso_search).shape

['rank_test_score', 'mean_test_score', 'std_test_score', 'params', 'mean_fit_time', 'std_fit_time']


(3, 6)

In [67]:
opt_lasso.score(X_test, y_test), opt_lasso_params

(0.7922897967423662, {'layer_sizes': [8, 8, 8, 8]})

In [77]:
from survwrap.metrics import *
scorers = {
    #'c-index-td': concordance_index_td_scorer,
    'neg-brier': make_time_dependent_scorer(neg_brier_score, time_mode='quantiles', time_values=[0.25, 0.5, 0.75]),
    'auc': make_time_dependent_scorer(roc_auc_td_score, time_mode='quantiles', time_values=[0.25, 0.5, 0.75]),
    'c-index-median': make_time_dependent_scorer(concordance_index_score, time_mode='quantiles', time_values=[0.5]),
}

#best_model, best_params, search_results = survwrap.optimize(deephit, X, y, mode='sklearn-random', user_grid=dict(dropout=[0.0, 0.2, 0.9]), scoring=scoring, tries=3, refit='c-index-td')

In [78]:
opt_lasso, opt_lasso_params, opt_lasso_search = survwrap.optimize(fl_lasso_cv,  X_train, y_train, 
                                                                  user_grid=fl_grid,
                                                                  scoring=scorers,
                                                                  refit='c-index-median',
                                                                  cv=survwrap.survival_crossval_splitter(X_train,y_train,
                                                                                                        n_repeats=1,n_splits=3),
                                                                 )

epoch: 0
loss: 7.9480061531066895
epoch: 1
loss: 7.943586826324463
epoch: 2
loss: 7.939194202423096
epoch: 3
loss: 7.934821605682373
epoch: 4
loss: 7.930466651916504
epoch: 5
loss: 7.92613410949707
epoch: 6
loss: 7.921833515167236
epoch: 7
loss: 7.9175848960876465
epoch: 8
loss: 7.913358211517334
epoch: 9
loss: 7.909154415130615
epoch: 10
loss: 7.9049811363220215
epoch: 11
loss: 7.900837421417236
epoch: 12
loss: 7.896726608276367
epoch: 13
loss: 7.892647743225098
epoch: 14
loss: 7.888603210449219
epoch: 15
loss: 7.8845977783203125
epoch: 16
loss: 7.880639553070068
epoch: 17
loss: 7.876720428466797
epoch: 18
loss: 7.872848987579346
epoch: 19
loss: 7.869026184082031
epoch: 20
loss: 7.865254878997803
epoch: 21
loss: 7.861538887023926
epoch: 22
loss: 7.857884407043457
epoch: 23
loss: 7.854283809661865
epoch: 24
loss: 7.850744724273682
epoch: 25
loss: 7.847270965576172
epoch: 26
loss: 7.843865871429443
epoch: 27
loss: 7.8405232429504395
epoch: 28
loss: 7.837236404418945
epoch: 29
loss: 7.83

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 117, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/usr/local/ivan/Unito/survwrap/survwrap/metrics.py", line 187, in scorer
    return score_func(y, y_pred, pred_times, **kwargs)
  File "/usr/local/ivan/Unito/survwrap/survwrap/metrics.py", line 11, in concordance_index_score
    r = sksurv.metrics.concordance_index_censored(
  File "/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sksurv/metrics.py", line 214, in concordance_index_censored
    event_indicator, event_time, estimate = _check_inputs(event_indicator, event_time, estimate)
  File "/usr/local/ivan/Unito/conda/envs/hive/lib

epoch: 0
loss: 7.980625629425049
epoch: 1
loss: 7.976261138916016
epoch: 2
loss: 7.971918106079102
epoch: 3
loss: 7.967597007751465
epoch: 4
loss: 7.963296890258789
epoch: 5
loss: 7.959023952484131
epoch: 6
loss: 7.9547834396362305
epoch: 7
loss: 7.9506096839904785
epoch: 8
loss: 7.946456432342529
epoch: 9
loss: 7.942324161529541
epoch: 10
loss: 7.9382171630859375
epoch: 11
loss: 7.9341325759887695
epoch: 12
loss: 7.9300689697265625
epoch: 13
loss: 7.926029682159424
epoch: 14
loss: 7.922010898590088
epoch: 15
loss: 7.918015480041504
epoch: 16
loss: 7.91403865814209
epoch: 17
loss: 7.910078525543213
epoch: 18
loss: 7.906129837036133
epoch: 19
loss: 7.902195930480957
epoch: 20
loss: 7.898271560668945
epoch: 21
loss: 7.894362449645996
epoch: 22
loss: 7.89046049118042
epoch: 23
loss: 7.886563777923584
epoch: 24
loss: 7.882673263549805
epoch: 25
loss: 7.878788471221924
epoch: 26
loss: 7.874904632568359
epoch: 27
loss: 7.871020317077637
epoch: 28
loss: 7.867135047912598
epoch: 29
loss: 7.863

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 117, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/usr/local/ivan/Unito/survwrap/survwrap/metrics.py", line 187, in scorer
    return score_func(y, y_pred, pred_times, **kwargs)
  File "/usr/local/ivan/Unito/survwrap/survwrap/metrics.py", line 11, in concordance_index_score
    r = sksurv.metrics.concordance_index_censored(
  File "/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sksurv/metrics.py", line 214, in concordance_index_censored
    event_indicator, event_time, estimate = _check_inputs(event_indicator, event_time, estimate)
  File "/usr/local/ivan/Unito/conda/envs/hive/lib

epoch: 0
loss: 7.965539932250977
epoch: 1
loss: 7.9612345695495605
epoch: 2
loss: 7.956954479217529
epoch: 3
loss: 7.952699661254883
epoch: 4
loss: 7.948470115661621
epoch: 5
loss: 7.944273471832275
epoch: 6
loss: 7.940122127532959
epoch: 7
loss: 7.936063289642334
epoch: 8
loss: 7.932027816772461
epoch: 9
loss: 7.928022384643555
epoch: 10
loss: 7.924047470092773
epoch: 11
loss: 7.92010498046875
epoch: 12
loss: 7.9161906242370605
epoch: 13
loss: 7.91231107711792
epoch: 14
loss: 7.908459186553955
epoch: 15
loss: 7.904630184173584
epoch: 16
loss: 7.900825500488281
epoch: 17
loss: 7.897034168243408
epoch: 18
loss: 7.893259048461914
epoch: 19
loss: 7.88949728012085
epoch: 20
loss: 7.885750770568848
epoch: 21
loss: 7.882022857666016
epoch: 22
loss: 7.878310203552246
epoch: 23
loss: 7.874615669250488
epoch: 24
loss: 7.870940685272217
epoch: 25
loss: 7.867282867431641
epoch: 26
loss: 7.863636493682861
epoch: 27
loss: 7.859999179840088
epoch: 28
loss: 7.85637092590332
epoch: 29
loss: 7.85274982

...

epoch: 0
loss: 8.7959566116333
epoch: 1
loss: 8.856557846069336
epoch: 2
loss: 8.831271171569824
epoch: 3
loss: 8.806096076965332
epoch: 4
loss: 8.781013488769531
epoch: 5
loss: 8.756102561950684
epoch: 6
loss: 8.731256484985352
epoch: 7
loss: 8.70648193359375
epoch: 8
loss: 8.681793212890625
epoch: 9
loss: 8.65733528137207
epoch: 10
loss: 8.633013725280762
epoch: 11
loss: 8.60880184173584
epoch: 12
loss: 8.584724426269531
epoch: 13
loss: 8.560802459716797
epoch: 14
loss: 8.537020683288574
epoch: 15
loss: 8.513387680053711
epoch: 16
loss: 8.489893913269043
epoch: 17
loss: 8.466545104980469
epoch: 18
loss: 8.443344116210938
epoch: 19
loss: 8.42029094696045
epoch: 20
loss: 8.397398948669434
epoch: 21
loss: 8.37466812133789
epoch: 22
loss: 8.352090835571289
epoch: 23
loss: 8.329673767089844
epoch: 24
loss: 8.307425498962402
epoch: 25
loss: 8.285356521606445
epoch: 26
loss: 8.263474464416504
epoch: 27
loss: 8.241759300231934
epoch: 28
loss: 8.220166206359863
epoch: 29
loss: 8.1987800598144

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 117, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/usr/local/ivan/Unito/survwrap/survwrap/metrics.py", line 187, in scorer
    return score_func(y, y_pred, pred_times, **kwargs)
  File "/usr/local/ivan/Unito/survwrap/survwrap/metrics.py", line 11, in concordance_index_score
    r = sksurv.metrics.concordance_index_censored(
  File "/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sksurv/metrics.py", line 214, in concordance_index_censored
    event_indicator, event_time, estimate = _check_inputs(event_indicator, event_time, estimate)
  File "/usr/local/ivan/Unito/conda/envs/hive/lib

epoch: 0
loss: 10.075550079345703
epoch: 1
loss: 10.445169448852539
epoch: 2
loss: 10.402426719665527
epoch: 3
loss: 10.359468460083008
epoch: 4
loss: 10.316298484802246
epoch: 5
loss: 10.272933006286621
epoch: 6
loss: 10.229354858398438
epoch: 7
loss: 10.18556022644043
epoch: 8
loss: 10.141554832458496
epoch: 9
loss: 10.097345352172852
epoch: 10
loss: 10.052946090698242
epoch: 11
loss: 10.008323669433594
epoch: 12
loss: 9.963456153869629
epoch: 13
loss: 9.918378829956055
epoch: 14
loss: 9.873096466064453
epoch: 15
loss: 9.827605247497559
epoch: 16
loss: 9.781903266906738
epoch: 17
loss: 9.735997200012207
epoch: 18
loss: 9.689888000488281
epoch: 19
loss: 9.643571853637695
epoch: 20
loss: 9.597040176391602
epoch: 21
loss: 9.55030345916748
epoch: 22
loss: 9.50336742401123
epoch: 23
loss: 9.4562349319458
epoch: 24
loss: 9.408894538879395
epoch: 25
loss: 9.36123275756836
epoch: 26
loss: 9.31342887878418
epoch: 27
loss: 9.26538372039795
epoch: 28
loss: 9.21712875366211
epoch: 29
loss: 9.168

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 117, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/usr/local/ivan/Unito/survwrap/survwrap/metrics.py", line 187, in scorer
    return score_func(y, y_pred, pred_times, **kwargs)
  File "/usr/local/ivan/Unito/survwrap/survwrap/metrics.py", line 11, in concordance_index_score
    r = sksurv.metrics.concordance_index_censored(
  File "/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sksurv/metrics.py", line 214, in concordance_index_censored
    event_indicator, event_time, estimate = _check_inputs(event_indicator, event_time, estimate)
  File "/usr/local/ivan/Unito/conda/envs/hive/lib

epoch: 0
loss: 8.370159149169922
epoch: 1
loss: 8.365800857543945
epoch: 2
loss: 8.361469268798828
epoch: 3
loss: 8.357159614562988
epoch: 4
loss: 8.352876663208008
epoch: 5
loss: 8.348620414733887
epoch: 6
loss: 8.344400405883789
epoch: 7
loss: 8.340253829956055
epoch: 8
loss: 8.336128234863281
epoch: 9
loss: 8.33202838897705
epoch: 10
loss: 8.327960014343262
epoch: 11
loss: 8.323922157287598
epoch: 12
loss: 8.319914817810059
epoch: 13
loss: 8.315943717956543
epoch: 14
loss: 8.312003135681152
epoch: 15
loss: 8.308094024658203
epoch: 16
loss: 8.304222106933594
epoch: 17
loss: 8.30038070678711
epoch: 18
loss: 8.296566009521484
epoch: 19
loss: 8.292780876159668
epoch: 20
loss: 8.289022445678711
epoch: 21
loss: 8.285293579101562
epoch: 22
loss: 8.281590461730957
epoch: 23
loss: 8.277910232543945
epoch: 24
loss: 8.27425765991211
epoch: 25
loss: 8.270627975463867
epoch: 26
loss: 8.267013549804688
epoch: 27
loss: 8.263416290283203
epoch: 28
loss: 8.259832382202148
epoch: 29
loss: 8.256258010

In [79]:
survwrap.get_model_scores_df(opt_lasso_search)

['rank_test_neg-brier', 'mean_test_neg-brier', 'std_test_neg-brier', 'rank_test_auc', 'mean_test_auc', 'std_test_auc', 'rank_test_c-index-median', 'mean_test_c-index-median', 'std_test_c-index-median', 'params', 'mean_fit_time', 'std_fit_time']


,rank_test_neg-brier,mean_test_neg-brier,std_test_neg-brier,rank_test_auc,mean_test_auc,std_test_auc,rank_test_c-index-median,mean_test_c-index-median,std_test_c-index-median,params,mean_fit_time,std_fit_time
0,1,-0.093461,0.002908,3,0.81575,0.016025,1,NaN,NaN,"{'layer_sizes': [8, 8]}",13.56029,0.48902
1,2,-0.093614,0.00262,2,0.81639,0.01445,1,NaN,NaN,"{'layer_sizes': [8, 8, 8]}",13.543517,0.405181
2,3,-0.094147,0.002876,1,0.817227,0.015398,1,NaN,NaN,"{'layer_sizes': [8, 8, 8, 8]}",14.749032,0.223881
